In [ ]:
import random as rd
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

#submit = True
submit = False

In [ ]:
# load the public MNIST dataset.
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_norm = x_train/255.
x_test_norm = x_test/255.

In [ ]:
#kaggle = pd.read_csv('../input/digit-recognizer/test.csv')
#kaggle_norm = np.asarray(kaggle/255.)
#kaggle_norm = kaggle_norm.reshape(28000, 28, 28, 1)

k_train = pd.read_csv('/content/train.csv')
k_labels = np.asarray(k_train['label'])
k_train = k_train.drop(columns=['label'])
k_train_norm = np.asarray(k_train/255.)

val_set_norm = k_train_norm[-2000:]
val_set_labels = k_labels[-2000:]
k_train_norm = k_train_norm[:-2000]
k_labels = k_labels[:-2000]

In [ ]:
# Add a channels dimension
k_train_norm = k_train_norm.reshape(40000, 28, 28, 1)
x_train_norm = x_train_norm.reshape(60000, 28, 28, 1)
x_test_norm = x_test_norm.reshape(10000, 28, 28, 1)
val_set_norm = val_set_norm.reshape(2000, 28, 28, 1)

In [ ]:
# train on the training set with some held back for validation #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=None):

    history = model.fit(x=train_features, y=train_label, 
                        batch_size=batch_size,
                        epochs=epochs, shuffle=True, 
                        validation_split=validation_split,
                        verbose = 1)

    # Gather the model's metrics after each round of training
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist

In [ ]:
 # Set up a CNN with Keras
 
def create_CNN(learning_rate):
    """Create and compile a convolutional neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, 4, activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, 2, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
# Train and evalate CNN on ALL the datasets, 
# Train on 10000 + 60000 + 42000 to predict on 28000
learning_rate = 0.001
epochs = 40
batch_size = 500
validation_split = 0.005 # 0.5% off bottom of each array
start_time = time.time()

x_trained = create_CNN(learning_rate)
x_tested = create_CNN(learning_rate)
k_trained = create_CNN(learning_rate)

#x_test_norm:
epochs_CNN1, hist_CNN1 = train_model(x_tested, x_test_norm, y_test, epochs, batch_size, validation_split)
print("\nFinished test set of 10000 in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#x_train_norm:
epochs_CNN2, hist_CNN2 = train_model(x_trained, x_train_norm, y_train, epochs, batch_size, validation_split)
print("\nFinished train set of 60000 in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#k_train_norm:
epochs_CNN3, hist_CNN3 = train_model(k_trained, k_train_norm, k_labels, epochs, batch_size, validation_split)
print("\nFinished kaggle set of 40000 in about {} seconds\n".format((round(time.time()-start_time,4))))

print ("\nTotal time: {} seconds is about {} minutes.".format(round(time.time()-start_time,4),
                                                              (time.time()-start_time)//60))

In [ ]:
if submit:
    
    def getKaggles():  
        kaggles = pd.DataFrame(columns=['ImageId','Label'])  
        predicts = convoluter.predict(kaggle_norm)
        for j in range(len(kaggle_norm)):
            probs = predicts[j] # one row of 10 probabilities 
            max_id = np.argmax(probs)   # index of top probability in row
            kaggles.at[j,'ImageId'] = j+1
            kaggles.at[j,'Label'] = max_id
        return kaggles
    
    kaggles = getKaggles()
    kaggles.to_csv('submission.csv', columns=["ImageId","Label"], index=False)

In [ ]:
def getPredictions(model=None, dataset=None): 
    ''' Takes a trained model and data
        Returns a List of guesses for that model on that data
    ''' 
    predictions = []
    predicts = model.predict(dataset)
    for j in range(len(dataset)):
        probs = predicts[j] # one row of 10 probabilities 
        max_id = np.argmax(probs)   # index of top probability in row
        predictions += [max_id]
    return predictions

In [ ]:
val_xtrain = getPredictions(model=x_trained, dataset=val_set_norm)
val_xtest = getPredictions(model=x_tested, dataset=val_set_norm)
val_ktrain = getPredictions(model=k_trained, dataset=val_set_norm)

In [ ]:
val_guesses = pd.DataFrame(columns=['label', 'xtrain', 'xtest', 'ktrain'])
val_guesses['label'] = val_set_labels
val_guesses['xtrain'] = val_xtrain
val_guesses['xtest'] = val_xtest
val_guesses['ktrain'] = val_ktrain
val_guesses

In [ ]:
tv0 = val_guesses['xtrain'] != val_guesses['label']
tv1 = val_guesses['xtest'] != val_guesses['label']
tv2 = val_guesses['ktrain'] != val_guesses['label']
tv3 = guesses['xtrain'] == guesses['xtest']
tv4 = guesses['xtrain'] == guesses['ktrain']
tv5 = guesses['ktrain'] == guesses['xtest']

#val_guesses.loc[lambda df: df['xtrain'] != df['label']]
# or df['xtest'] != df['label']or df['ktrain'] != df['label']]

In [ ]:
guesses = val_guesses.loc[tv0==True]
guesses = guesses.append(val_guesses.loc[tv1==True])
guesses = guesses.append(val_guesses.loc[tv2==True])
guesses = guesses.drop_duplicates()
guesses

In [ ]:
val_guesses.loc[tv0==True]

In [ ]:
guesses.loc[tv3==True]

In [ ]:
guesses.loc[tv4==True]

In [ ]:
guesses.loc[tv5==True]

##Visualizations

In [ ]:
if not submit:
# Plot a graph of the 'accuracy' metric vs. epochs:
    plt.plot(epochs_CNN3,hist_CNN3["accuracy"])
    plt.plot(epochs_CNN3,hist_CNN3["val_accuracy"])